## Instalação das bibliotecas

In [1]:
# !pip install sentence_transformers pypdf faiss-gpu
# !pip install langchain langchain-openai
%pip install --upgrade --quiet langchain langchain-community langchain-huggingface langchain-openai # text-generation transformers langchainhub sentencepiece jinja2
%pip install --upgrade --quiet sentence_transformers pypdf faiss-gpu

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

#%pip install --upgrade --quiet bitsandbytes accelerate

# Importação de bibliotecas
import os
import json
import zipfile
import re
import random
from google.colab import files

from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

## Funções auxiliares

In [2]:
# Function to perform some generic operation on a file
def process_file(file_path, data):
    print(f"Processing {file_path}")
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    category = os.path.basename(file_path).split(".pdf")[0]
    data[category] = pages

def extract_zip(zip_file_path, extract_dir):
    # Unzip the file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    dataset = {}
    folder_path = extract_dir + "/" + os.listdir(extract_dir)[0]
    files = sorted(os.listdir(folder_path))

    # Iterate through the files in the extracted directory
    for pdf in files:
        file_path = os.path.join(folder_path, pdf)
        process_file(file_path, dataset)

    return dataset

## Carrega modelo de embeddings e tinyllama

In [3]:
# Load embedding model
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5",
encode_kwargs={"normalize_embeddings": True})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
model_tiny_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

baseline_model = HuggingFaceEndpoint(
    repo_id=model_tiny_name,
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.15,
)

## Criar o RAG através do arquivo de dados
Caso não tenha o arquivo de RAG pronto (.db), podemos cria-lo usando o arquivo de dados da lesgislação.

**PARA USAR A SEGUINTE CÉLULA, DEVE-SE TER O ARQUIVO: "lesgislação_dataset.zip" NO DIRETÓRIO "/content" DO COLAB.**


In [ ]:
# Retira retira dataset do arquivo zip
zip_file_path = '/content/legislacao_dataset.zip'
extract_dir = '/content/data'

dataset = extract_zip(zip_file_path, extract_dir)

# Carrega o conteudo do database
pdfs = os.listdir('/content/data/lesgilacao_dataset')

documents = []
for pdf in pdfs:
    print(pdf)
    loader = PyPDFLoader('/content/data/lesgilacao_dataset/'+pdf)
    documents.extend(loader.load())

# Divide em chunks e gera os embeddings
text = RecursiveCharacterTextSplitter(chunk_size = 1000).split_documents(documents)

# Faz a indexação e cria o arquivo do RAG
# Create a vectorstore
vectorstore = FAISS.from_documents(text, embeddings)
# Save the documents and embeddings
vectorstore.save_local("vectorstore_1000_chunks.db")
# Create retriever
retriever = vectorstore.as_retriever()

In [ ]:
def criar_arquivo_zip(pasta_origem, nome_arquivo_zip):
    with zipfile.ZipFile(nome_arquivo_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for raiz, dirs, arquivos in os.walk(pasta_origem):
            for arquivo in arquivos:
                caminho_completo = os.path.join(raiz, arquivo)
                zipf.write(caminho_completo, os.path.relpath(caminho_completo, pasta_origem))

pasta_origem = '/content/vectorstore_1000_chunks.db'
nome_arquivo_zip = 'vectorstore_1000_chunks.db.zip'

criar_arquivo_zip(pasta_origem, nome_arquivo_zip)


## Acessar RAG através do database vectorstore

Caso já possua o arquivo de RAG pronto, podemos carrega-lo.

**PARA USAR A SEGUINTE CÉLULA DEVE-SE TER O ARQUIVO "vectorstore_1000_chunks.db.zip"NO DIRETÓRIO "/content" DO COLAB.**

In [5]:
# Extrai o database
zip_file_path = '/content/vectorstore_1000_chunks.db.zip'
extract_dir = '/content/vectorstore_1000_chunks.db'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# carrega o database
vectorstore = FAISS.load_local("vectorstore_1000_chunks.db", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

## Chain com RAG

In [24]:
# Define prompt template
template = """
Você é um assistente para tarefas de perguntas e respostas.
Use o contexto abaixo como fonte exclusiva para responder a pergunta que deve ser respondida:

<cotext>
{context}
</context>

Pergunta que deve ser respondida: {input}

Resposta esperada:
"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)

# Create a chain
doc_chain = create_stuff_documents_chain(baseline_model, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

## Testes e avaliação

Para fazer os testes usando o RAG, contruímos uma série de perguntas baseadas em artigos selecionados da base de dados.

- Primeiro escolhemos 5 assuntos (carteira estudantil, Programa de Educação Tutorial, Estatuto da ufam, Educação à distância, curso de férias).
- Depois elaboramos perguntas com base em regulamentações, as respostas corretas devem ser interpretadas como o conceito dessas regulamentações.

#### Perguntas e respostas esperadas:

```
Assunto: carteira estudantil, artigo 1
Pergunta: Em que eventos os alunos podem entrar com meia entrada usando a carteira estudantil?
Resposta:
1o** É assegurado aos estudantes o acesso a salas de cinema, cineclubes, teatros, espetáculos
musicais e circenses e eventos educativos, esportivos, de lazer e de entretenimento


Assunto : Programa de Educação Tutorial
Pergunta: Qual a definição do PET? também conhecida por Programa de Educação Tutorial
Resposta:
**Art. 2o** O PET constitui-se em programa de educação tutorial desenvolvido em grupos organizados a
partir de cursos de graduação das instituições de ensino superior do País, orientados pelo princípio da
indissociabilidade entre ensino, pesquisa e extensão, com os seguintes objetivos:
I — Desenvolver atividades acadêmicas em padrões de qualidade de excelência, mediante grupos de
aprendizagem tutorial de natureza coletiva e interdisciplinar;
II — Contribuir para a elevação da qualidade da formação acadêmica dos alunos de graduação;
III — Estimular a formação de profissionais e docentes de elevada qualificação técnica, científica,
tecnológica e acadêmica;
IV — Formular novas estratégias de desenvolvimento e modernização do ensino superior no país;
V — Estimular o espírito crítico, bem como a atuação profissional pautada pela cidadania e pela função
social da educação superior.



Assunto: Estatuto da ufam
Pergunta: Quais são modalidades que a universidade pode ministrar?
Resposta:
I — sequenciais, por campo de saber, de diferentes níveis de abrangência, abertos a candidatos que
atendam aos requisitos estabelecidos no Regimento Geral;
II — de graduação, abertos a candidatos que tenham concluído o ensino médio ou equivalente e tenham
sido classificados em processo seletivo;
III — de pós-graduação, compreendendo programas de mestrado e doutorado, cursos de especialização,
aperfeiçoamento e outros, abertos a candidatos diplomados em cursos de graduação e que atendam às
exigências do Regimento Geral e normas complementares;
IV — de extensão, abertos a candidatos que atendam aos requisitos estabelecidos pela Universidade.


Assunto: Educação à distância
Pergunta: Com relação a educação a distância, os cursos a distância podem aceitar transferência de cursos presenciais?
Resposta:
**Art. 4o** Os cursos a distância poderão aceitar transferência e aproveitar créditos obtidos pelos alunos
em cursos presenciais, da mesma forma que as certificações totais ou parciais obtidas em cursos a
distância poderão ser aceitas em cursos presenciais.


Assunto: curso de férias
Pergunta: Quantos cursos no máximo um aluno pode cursar mas férias?
Resposta:
**Art. 4o** O aluno poderá cursar, no máximo, 2 (duas) disciplinas em período especial, de
forma consecutiva, respeitados os pré-requisitos.
I — Ao cursar 2 (duas) disciplinas, estas não deverão ter mais que 80% da carga horária
cursada concomitantemente e não poderá haver dependência de pré-requisitos entre elas.
II — Não será permitida a matrícula do aluno durante ou após a realização da disciplina em
período especial, conforme resolução n° 026/2007-CONSKPE.

```



### Execução das perguntas

In [26]:
teste1 = chain.invoke({"input": "Em que eventos os alunos podem entrar com meia entrada usando a carteira estudantil?"})
teste2 = chain.invoke({"input": "Qual a definição do PET? também conhecida por Programa de Educação Tutorial"})
teste3 = chain.invoke({"input": "Quais são modalidades que a universidade pode ministrar?"})
teste4 = chain.invoke({"input": "Com relação a educação a distância, os cursos a distância podem aceitar transferência de cursos presenciais?"})
teste5 = chain.invoke({"input": "Quantos cursos no máximo um aluno pode cursar mas férias?"})

In [17]:
resp1 = '''1o** É assegurado aos estudantes o acesso a salas de cinema, cineclubes, teatros, espetáculos
musicais e circenses e eventos educativos, esportivos, de lazer e de entretenimento'''

resp2 = '''**Art. 2o** O PET constitui-se em programa de educação tutorial desenvolvido em grupos organizados a
partir de cursos de graduação das instituições de ensino superior do País, orientados pelo princípio da
indissociabilidade entre ensino, pesquisa e extensão, com os seguintes objetivos:
I — Desenvolver atividades acadêmicas em padrões de qualidade de excelência, mediante grupos de
aprendizagem tutorial de natureza coletiva e interdisciplinar;
II — Contribuir para a elevação da qualidade da formação acadêmica dos alunos de graduação;
III — Estimular a formação de profissionais e docentes de elevada qualificação técnica, científica,
tecnológica e acadêmica;
IV — Formular novas estratégias de desenvolvimento e modernização do ensino superior no país;
V — Estimular o espírito crítico, bem como a atuação profissional pautada pela cidadania e pela função
social da educação superior.'''

resp3 = '''I — sequenciais, por campo de saber, de diferentes níveis de abrangência, abertos a candidatos que
atendam aos requisitos estabelecidos no Regimento Geral;
II — de graduação, abertos a candidatos que tenham concluído o ensino médio ou equivalente e tenham
sido classificados em processo seletivo;
III — de pós-graduação, compreendendo programas de mestrado e doutorado, cursos de especialização,
aperfeiçoamento e outros, abertos a candidatos diplomados em cursos de graduação e que atendam às
exigências do Regimento Geral e normas complementares;
IV — de extensão, abertos a candidatos que atendam aos requisitos estabelecidos pela Universidade.'''

resp4 = '''**Art. 4o** Os cursos a distância poderão aceitar transferência e aproveitar créditos obtidos pelos alunos
em cursos presenciais, da mesma forma que as certificações totais ou parciais obtidas em cursos a
distância poderão ser aceitas em cursos presenciais.'''

resp5 = '''**Art. 4o** O aluno poderá cursar, no máximo, 2 (duas) disciplinas em período especial, de
forma consecutiva, respeitados os pré-requisitos.
I — Ao cursar 2 (duas) disciplinas, estas não deverão ter mais que 80% da carga horária
cursada concomitantemente e não poderá haver dependência de pré-requisitos entre elas.
II — Não será permitida a matrícula do aluno durante ou após a realização da disciplina em
período especial, conforme resolução n° 026/2007-CONSKPE.'''

In [18]:
def testa(test, resp):
    print(f'''Pergunta: {test['input']}\n\nResposta esperada: {resp}\n\nResposta do modelo: {test['answer']}\n\n\nContexto: {test['context']}''')

In [27]:
testa(teste1, resp1)

Pergunta: Em que eventos os alunos podem entrar com meia entrada usando a carteira estudantil?

Resposta esperada: 1o** É assegurado aos estudantes o acesso a salas de cinema, cineclubes, teatros, espetáculos
musicais e circenses e eventos educativos, esportivos, de lazer e de entretenimento

Resposta do modelo: Para os eventos de acordo com o artigo 5º desta lei, o beneficio da meia-entrada para acesso a eventos 
artísticos, culturais e esportivos por jovens de baixa renda, por estudantes e por pessoas com deficiência, 
é reguleado nas disposições deste decreto. Esses eventos incluem, entre outros, os veículos do sistema de 
transporte coletivo interestadual. Portanto, os alunos podem usar a carteira estudantil para entrar com meia 
entrada aos veículos relacionados a esses eventos.


Contexto: [Document(metadata={'source': '/content/data/lesgilacao_dataset/Carteira Estudantil.pdf', 'page': 0}, page_content='# Assunto : Carteira Estudantil  \n \n**Art. 1º** É assegurado aos estudantes

In [28]:
testa(teste2, resp2)

Pergunta: Qual a definição do PET? também conhecida por Programa de Educação Tutorial

Resposta esperada: **Art. 2o** O PET constitui-se em programa de educação tutorial desenvolvido em grupos organizados a
partir de cursos de graduação das instituições de ensino superior do País, orientados pelo princípio da
indissociabilidade entre ensino, pesquisa e extensão, com os seguintes objetivos:
I — Desenvolver atividades acadêmicas em padrões de qualidade de excelência, mediante grupos de
aprendizagem tutorial de natureza coletiva e interdisciplinar;
II — Contribuir para a elevação da qualidade da formação acadêmica dos alunos de graduação;
III — Estimular a formação de profissionais e docentes de elevada qualificação técnica, científica,
tecnológica e acadêmica;
IV — Formular novas estratégias de desenvolvimento e modernização do ensino superior no país;
V — Estimular o espírito crítico, bem como a atuação profissional pautada pela cidadania e pela função
social da educação superior.

Resp

In [29]:
testa(teste3, resp3)

Pergunta: Quais são modalidades que a universidade pode ministrar?

Resposta esperada: I — sequenciais, por campo de saber, de diferentes níveis de abrangência, abertos a candidatos que
atendam aos requisitos estabelecidos no Regimento Geral;
II — de graduação, abertos a candidatos que tenham concluído o ensino médio ou equivalente e tenham
sido classificados em processo seletivo;
III — de pós-graduação, compreendendo programas de mestrado e doutorado, cursos de especialização,
aperfeiçoamento e outros, abertos a candidatos diplomados em cursos de graduação e que atendam às
exigências do Regimento Geral e normas complementares;
IV — de extensão, abertos a candidatos que atendam aos requisitos estabelecidos pela Universidade.

Resposta do modelo: - Diplomas de graduação e de Mestrado e Doctorado concedidos por universidades extranjeras
para estudantes que atingam a condição necessária, em âmbito cumulativo comuns às duas áreas de 
conhecimento e em nível equivalente ou superior.
- Permi

In [30]:
testa(teste4, resp4)

Pergunta: Com relação a educação a distância, os cursos a distância podem aceitar transferência de cursos presenciais?

Resposta esperada: **Art. 4o** Os cursos a distância poderão aceitar transferência e aproveitar créditos obtidos pelos alunos
em cursos presenciais, da mesma forma que as certificações totais ou parciais obtidas em cursos a
distância poderão ser aceitas em cursos presenciais.

Resposta do modelo: Os cursos a distância podem aceitar transferência de cursos presenciais quando for o caso, mas há requisitos especiais para isso. Deve-se monitorar a transferência de cursos presenciais para verificar se eles estiverem satisfatíssimos nas expectativas escolares.


Contexto: [Document(metadata={'source': '/content/data/lesgilacao_dataset/Educaç╞o a Distância.pdf', 'page': 0}, page_content='em cursos presenciais, da mesma forma que as certificações totais ou parciais obtidas em cursos a \ndistância poderão ser aceitas em cursos presenciais.  \n \n**Art. 5º** Os certificados e d

In [31]:
testa(teste5, resp5)

Pergunta: Quantos cursos no máximo um aluno pode cursar mas férias?

Resposta esperada: **Art. 4o** O aluno poderá cursar, no máximo, 2 (duas) disciplinas em período especial, de
forma consecutiva, respeitados os pré-requisitos.
I — Ao cursar 2 (duas) disciplinas, estas não deverão ter mais que 80% da carga horária
cursada concomitantemente e não poderá haver dependência de pré-requisitos entre elas.
II — Não será permitida a matrícula do aluno durante ou após a realização da disciplina em
período especial, conforme resolução n° 026/2007-CONSKPE.

Resposta do modelo: - Nunca



Contexto: [Document(metadata={'source': '/content/data/lesgilacao_dataset/Matrícula.pdf', 'page': 4}, page_content='cursos em todos os períodos de solicitação de matrícula.  \n \n**§ 2º** Para todos os critérios de seleção, o desempate se efetuará por meio do \ncoeficiente de rendim ento.  \n \n**Art. 11** A matrícula em disciplina(s)/turma(s) será realizada estritamente dentro dos \nlimites mínimo e máximo de c